In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
with open('email_training_data.json') as file:
    training_data_json = json.load(file)
    df_train = pd.DataFrame(training_data_json)

with open('email_testing_data.json') as file:
    testing_data_json = json.load(file)
    df_test = pd.DataFrame(testing_data_json)

with open('emaildata.json') as file:
    data = json.load(file)
    df = pd.DataFrame(data)

In [16]:
df_train_y = df_train[['Category']].copy()
df_train_X = df_train[['Content']].copy()
df_test_y = df_test[['Category']].copy()
df_test_X = df_test[['Content']].copy()
df_train_y = np.squeeze(df_train_y)
df_test_y = np.squeeze(df_test_y)
df_train_X = np.squeeze(df_train_X)
df_test_X = np.squeeze(df_test_X)
df_train_y.shape, df_train_X.shape, df_test_y.shape, df_test_X.shape

((144,), (144,), (36,), (36,))

In [1]:
from sklearn.model_selection import train_test_split

df_train_X, df_test_X, df_train_y, df_test_y = train_test_split(
    df['Content'], df['Category'], test_size=0.2)

NameError: name 'df' is not defined

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=1000, ngram_range=(1, 2))
X_train_count = count_vectorizer.fit_transform(df_train_X)
X_test_count = count_vectorizer.transform(df_test_X)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_count, df_train_y)
predictions = classifier.predict(X_test_count)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6111111111111112


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 1000, ngram_range = (1,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train_X)
X_test_tfidf = tfidf_vectorizer.transform(df_test_X)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, df_train_y)
predictions = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6666666666666666


In [2]:
X_train_count = X_train_count.toarray()
X_test_count = X_test_count.toarray()
X_train_count = np.concatenate([X_train_count, df_train_X.apply(len).values.reshape(-1, 1), df_train_X.apply(lambda x: len(x.split())).values.reshape(-1, 1)], axis=1)
X_test_count = np.concatenate([X_test_count, df_test_X.apply(len).values.reshape(-1, 1), df_test_X.apply(lambda x: len(x.split())).values.reshape(-1, 1)], axis=1)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_count, df_train_y)
predictions = classifier.predict(X_test_count)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

NameError: name 'X_train_count' is not defined

In [8]:
import gensim

model = gensim.models.Word2Vec.load('word2vec_imdb.model')
def word_averaging(wv, words):
    try:
        vectors = [wv[word] for word in words if word in wv.key_to_index]
        return gensim.matutils.unitvec(np.mean(vectors, axis=0)).astype(np.float32)
    except Exception as e:
        print(e, words) # some words are not in the vocabulary
        return np.zeros(wv.vector_size)

X_train_word2vec = np.array(
    [word_averaging(model.wv, message.split()) for message in df_train_X])
X_test_word2vec = np.array(
    [word_averaging(model.wv, message.split()) for message in df_test_X])

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_word2vec, df_train_y)
predictions = classifier.predict(X_test_word2vec)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

TypeError: __randomstate_ctor() takes from 0 to 1 positional arguments but 2 were given